2025-10-17 23:57:58 Friday


### 1.在大模型训练中，LoRA（Low-Rank Adaptation） 的原理是什么？为什么它能在保持性能的同时大幅减少参数量？
大模型预训练权重已经包含了通用知识，微调时只需要对权重矩阵做任务特定的稀疏调整（ΔW）。LoRA通过低秩矩阵A和B来逼近这个更新（ΔW≈A·B），这类似于用可训练参数实现了一种数据驱动的低秩近似，而不需要显式计算SVD分解。

### 2.请解释一下 进程（Process） 与 线程（Thread） 的区别。并说明为什么多线程共享内存能提高效率，但也可能引发并发问题。

- 进程 是操作系统进行资源分配和保护的基本单位，每个进程有独立的内存空间；

- 线程 是 CPU 调度的最小执行单元，同一进程中的线程共享进程的资源；

- 多线程优势：减少上下文切换成本、共享数据传输快；

- 风险：需显式同步，否则会造成 race condition 或死锁，且任一线程异常会导致整个进程终止。

### 3.在多模态模型（如 CLIP）中，图像和文本如何在同一语义空间中对齐？请你说明训练目标（loss）是如何设计的。

CLIP 通过独立的图像编码器和文本编码器，把图像和文本都映射到同一个语义空间。
在一个 batch 里，我们计算所有图文对的相似度矩阵 $S_{ij} = (v_i \cdot t_j)/\tau$。
然后分别做两个方向的 Softmax + Cross-Entropy：

再分别计算两个方向的交叉熵，最后求两者平均获得损失，即$L = \frac{1}{2}(CE(s,labels) + CE(s^T,labels))$

伪代码描述为
```
# logits: [B, B] = (v @ t.T) / tau，v/t 已做 L2 归一化
labels = torch.arange(B, device=logits.device)
loss_i2t = F.cross_entropy(logits, labels)       # 行为图像，列为文本
loss_t2i = F.cross_entropy(logits.T, labels)     # 行为文本，列为图像
loss = 0.5 * (loss_i2t + loss_t2i)

故而CLIP 的对齐过程就是对相似度矩阵做双向交叉熵（本质为 InfoNCE），
让匹配图文对的相似度更高、非匹配对更低，从而学习到共享的语义空间。